In [27]:
import keras.layers
from tensorflow import keras
import tensorflow as tf

In [19]:
# Path to images
path_to_images = "training-data/dataset1/known_images"

In [30]:
train_dataset, val_dataset = keras.utils.image_dataset_from_directory(
    path_to_images,
    class_names=["normal", "sick"],
    batch_size = 32,
    image_size=(395, 488),
    validation_split=0.2,
    subset="both",
    seed=200
)

Found 715 files belonging to 2 classes.
Using 572 files for training.
Using 143 files for validation.


In [62]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomZoom(0.2)
    ]
)

In [67]:
# Create input shape
inputs = keras.Input(shape=(395, 488, 3))

# Data augmentation
x = data_augmentation(inputs)

# Normalize the features from (0 -255) to (0 -1)
x = keras.layers.Rescaling(1./255)(x)

# Create a convolutional layer with max pooling
x = keras.layers.Conv2D(filters=32,kernel_size=3,activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=2)(x)

x = keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu")(inputs)
x = keras.layers.MaxPooling2D(pool_size=2)(x)

x = keras.layers.Conv2D(filters=128,kernel_size=3,activation="relu")(inputs)
x = keras.layers.MaxPooling2D(pool_size=2)(x)

x = keras.layers.Conv2D(filters=256,kernel_size=3,activation="relu")(inputs)
x = keras.layers.MaxPooling2D(pool_size=2)(x)

x = keras.layers.Conv2D(filters=256,kernel_size=3,activation="relu")(inputs)
x = keras.layers.MaxPooling2D(pool_size=2)(x)

x = keras.layers.Conv2D(filters=256,kernel_size=3,activation="relu")(inputs)
x = keras.layers.MaxPooling2D(pool_size=2)(x)

x = keras.layers.Flatten()(x)

# Dropout
x = keras.layers.Dropout(0.5)(x)

# Create the model output
outputs = keras.layers.Dense(1, activation="sigmoid")(x)


In [68]:
model = keras.Model(inputs=inputs, outputs=outputs, name="Chest_Opacities_Detection")

In [69]:
model.summary()

Model: "Chest_Opacities_Detection"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 395, 488, 3)]     0         
                                                                 
 conv2d_54 (Conv2D)          (None, 393, 486, 256)     7168      
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 196, 243, 256)    0         
 g2D)                                                            
                                                                 
 flatten_12 (Flatten)        (None, 12192768)          0         
                                                                 
 dropout (Dropout)           (None, 12192768)          0         
                                                                 
 dense_12 (Dense)            (None, 1)                 12192769  
                                         

In [70]:
model.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=['accuracy']
)

In [71]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="models/chest_opacities_detection_model_V1.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

In [72]:
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=val_dataset,
    callbacks=callbacks
)

Epoch 1/5
18/18 [==============================] - 494s 26s/step - loss: 2678.1025 - accuracy: 0.7063 - val_loss: 296.0400 - val_accuracy: 0.5524
Epoch 2/5
18/18 [==============================] - 422s 23s/step - loss: 51.3132 - accuracy: 0.9126 - val_loss: 13.0102 - val_accuracy: 0.8811
Epoch 3/5
10/18 [===============>..............] - ETA: 2:50 - loss: 9.4121 - accuracy: 0.9156

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training Accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation Accuracy")
plt.title("Training and Validation accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label="Training Loss")
plt.plot(epochs, val_loss, "b", label="Validation Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.figure()
plt.show()